# Regression of Vote Characteristics on CAR

In [25]:
import stata_setup

stata_setup.config('/Applications/StataNow', 'mp')

In [26]:
%%stata

clear all
set more off
version 18

* Paths
global PROCESSED_DATA "processed_data"
global TABLES "tables"

In [ ]:
%%stata
foreach stage in created end {
    import delimited using "$PROCESSED_DATA/event_study_panel_`stage'.csv", clear

    * Parse date
    gen day = date(date, "YMD")
    format day %td
    gen year = year(day)

    * Ensure index is numeric (safeguard if it came in as string)
    capture confirm numeric variable index
    if _rc {
        destring index, replace ignore("[]")
    }

    * Encode categorical variables
    encode gecko_id, gen(token)
    encode space,    gen(dao)

    * Define windows and specs
    local windows 2 5
    local vars    half_vp_ratio vn_hhi vs_hhi cci

    eststo clear

    foreach w of local windows {
        preserve
        keep if index == `w'

        foreach v of local vars {
            qui reghdfe car_`w' `v', absorb(dao year) vce(cluster dao)
            eststo car`w'_`v'
        }

        restore
    }

    * Export a single big LaTeX table with 8 models (2 windows × 4 specs)
    esttab                                                          ///
        car2_half_vp_ratio car2_vn_hhi car2_vs_hhi car2_cci        ///
        car5_half_vp_ratio car5_vn_hhi car5_vs_hhi car5_cci        ///
        using "$TABLES/reg_car_votes_`stage'.tex", replace          ///
        se star(* 0.10 ** 0.05 *** 0.01) label nogaps               ///
        mtitles("Half VP" "VN HHI" "VS HHI" "CCI"                  ///
                "Half VP" "VN HHI" "VS HHI" "CCI")                 ///
        mgroups("CAR[-2,2]" "CAR[-5,5]",                            ///
            span                                                   ///
            pattern(1 1 1 1 0 0 0 0)                               ///
            prefix(\multicolumn{@span}{c}{)                        ///
            suffix(}))                                             ///
        stats(N r2_a, fmt(0 3) labels("N" "Adj. R²"))
}